In [1]:
# %pip install llama-index llama-index-core llama-parse openai llama_index.embeddings.huggingface -q
# %pip install llama-index-llms-anthropic -q
# %pip install llama-index-vector-stores-weaviate -q

In [4]:
COMPARISON_FILE = 'claude-3-5-sonnet-20240620_qa.csv'
PDF_LOCATION = 'IndustrySource/Misc/62 Healthcare and Social Assistance in the US Industry Report.pdf'
DOC_ID = 'ibis-healthcare-social-assistance'
MODEL_ID = 'gpt-4o-mini'
OUTPUT_FILE = f'{DOC_ID}_{MODEL_ID}.csv'
QUESTION_COL = 'question'
RESPONSE_COL = 'rag_model_response'
NUM_QUESTIONS = 25

In [5]:
import pandas as pd

df = pd.read_csv(COMPARISON_FILE)
dff = df.head(NUM_QUESTIONS).copy()

In [7]:
# llama-parse is async-first, running the async code in a notebook requires the use of nest_asyncio
import nest_asyncio
nest_asyncio.apply()

import os
from dotenv import load_dotenv
load_dotenv()

# API access to llama-cloud
os.environ["LLAMA_CLOUD_API_KEY"] = "llx-7K1IbMcLbyb8TDvMsIx3Brr7mD4K8ZnLaFMjbEq8S1uONYZp"
# Using OpenAI API for embeddings/llms
# os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_APIKEY')
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [8]:
# from llama_index.core import SimpleDirectoryReader

from llama_parse import LlamaParse
documents = LlamaParse(result_type="markdown").load_data(PDF_LOCATION)
len(documents)

Started parsing the file under job_id c922a8e0-aba6-41f6-96da-3f7975498489


In [16]:
import uuid

# generate unique index for multiple runs
INDEX_NAME = ('X' + str(uuid.uuid4())).replace('-', '_')

In [17]:
import weaviate

cluster_url = "https://a0dlgmcaspopjrn2mtx4ha.c0.us-east1.gcp.weaviate.cloud"
api_key = "7ZfUCibywHnzM0WKMPx7YevuN79nUtS4KJgT"

client = weaviate.connect_to_wcs(
    cluster_url=cluster_url,
    auth_credentials=weaviate.auth.AuthApiKey(api_key),
)

# weaviate vector database & llamaparse Integrated

In [44]:
print(documents[20].text)
print(len(documents))

# IBISWorld | Healthcare and Social Assistance in the US

# Mar 2024

- The dramatic rise in the adoption of telemedicine in 2020 has fallen from its pandemic high but is still more integral to healthcare delivery in 2024 than pre-COVID, especially for providers offering mental health treatment.
- The challenges impacting healthcare providers are also expediting the adoption of other digital tools (AI, patient engagement products, data analytics) to streamline operations and produce cost savings.

# Social assistance providers fill in the gaps as pandemic-era benefits end

- The pandemic highlighted societal inequities. Social assistance providers have had an outsized role in addressing these inequities since the COVID-19 pandemic subsided.
- Pandemic relief funding that cushioned savings accounts and lifted many out of poverty ended between 2022 and 2023, with exact end dates varying between states. As benefits disappeared, millions turned to social assistance providers for help with 

In [53]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import TextNode


splitter = SentenceSplitter(chunk_size=1024)
nodes = []
for idx, doc in enumerate(documents):
    chunks = splitter.split_text(doc.text)
    # create nodes with metadata
    for i, chunk in enumerate(chunks):
        node = TextNode(text=chunk, metadata={'page_number': idx+1})
        nodes.append(node)


print(len(nodes))


81
{'page_number': 1}


In [54]:
from llama_index.vector_stores.weaviate import WeaviateVectorStore

vector_store = WeaviateVectorStore(
    weaviate_client=client, index_name=INDEX_NAME
)

In [55]:
from llama_index.core import VectorStoreIndex
vector_index = VectorStoreIndex(nodes, vector_store = vector_store)

In [56]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model=MODEL_ID, api_key = OPENAI_API_KEY)
query_engine = vector_index.as_query_engine(similarity_top_k=3, llm=llm)

In [72]:
result = []
references = []
for question in dff[QUESTION_COL]:
    result.append((query_engine.query(question).response))
    metadata = query_engine.query(question).metadata
    refs = []
    for m in metadata.values():
        refs.append(m['page_number'])
    references.append(refs)

dff[RESPONSE_COL] = result
dff['PageNumberReferences'] = references
OUTPUT_FILE = f'{DOC_ID}_{MODEL_ID}_testing.csv'
dff.to_csv(OUTPUT_FILE, index=False)